In [4]:
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.linear_model import LinearRegression


drive.mount('/content/drive')
housing_csv_url = "/content/drive/MyDrive/Colab Notebooks/Housing.csv"
df = pd.read_csv(housing_csv_url)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [5]:
df.describe()

,price,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,545.000000,545.000000,545.000000,545.000000,545.000000
mean,4.766729e+06,5150.541284,2.965138,1.286239,1.805505,0.693578
std,1.870440e+06,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1.750000e+06,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3.430000e+06,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4.340000e+06,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,5.740000e+06,6360.000000,3.000000,2.000000,2.000000,1.000000
max,1.330000e+07,16200.000000,6.000000,4.000000,4.000000,3.000000


In [7]:

#напишіть функцію гіпотези лінійної регресії у векторному вигляді;

def hypothesis(w, X):
    return X @ w

In [8]:

#створіть функцію для обчислення функції втрат у векторному вигляді;

def cost(y, h):
    return np.mean((h - y) ** 2) / 2

In [10]:
#реалізуйте один крок градієнтного спуску;

# def gradient_descent(X, y, w, lr):
#     h = X @ w
#     gradient = X.T @ (h - y)
#     w -= lr * gradient
#     return w

def gradient_descent(X, y, max_iters=10000, learning_rate=0.001, stopping_threshold=1e-6):
    X = np.c_[np.ones(X.shape[0]), X]
    w = np.zeros(X.shape[1])
    n = float(len(X))

    costs_list = []
    weights = []
    previous_cost = None

    for i in range(max_iters):
        h = X @ w
        costs = cost(y, h)

        if previous_cost is not None and abs(previous_cost - costs) <= stopping_threshold:
            break
        previous_cost = costs

        costs_list.append(costs)
        weights.append(w[1])

        gradient = X.T @ (h - y)
        w -= learning_rate * gradient / n

    return w

In [11]:
#знайдіть ці ж параметри за допомогою аналітичного рішення;

def analytical_linear_regression(X, y):
    X = np.c_[np.ones(X.shape[0]), X]
    w = np.linalg.inv(X.T @ X) @ X.T @ y
    return w

In [15]:
#порівняйте отримані результати.

X = df[['area', 'bathrooms', 'bedrooms']]
y = df['price'] / 1000000
X.loc[:, 'area'] = X.loc[:, 'area'] / 1000

custom_w = gradient_descent(X, y)
lr = LinearRegression().fit(X, y)
analytical_w = analytical_linear_regression(X, y)

print('{:^30}|{:^30}|{:^30}|{:^30}'.format('feature', 'custom_LR_method', 'sklearn_LR_method', 'analytic_method'))
print('-'*120)
print('{:^30}|{:^30}|{:^30}|{:^30}'.format('area', custom_w[1], lr.coef_[0], analytical_w[1]))
print('-' * 120)
print('{:^30}|{:^30}|{:^30}|{:^30}'.format('bathrooms', custom_w[2], lr.coef_[1], analytical_w[2]))
print('-'*120)
print('{:^30}|{:^30}|{:^30}|{:^30}'.format('bedrooms', custom_w[3], lr.coef_[2], analytical_w[3]))

<ipython-input-15-0d38718a5031>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, 'area'] = X.loc[:, 'area'] / 1000


           feature            |       custom_LR_method       |      sklearn_LR_method       |       analytic_method        
------------------------------------------------------------------------------------------------------------------------
             area             |     0.37467319694812257      |     0.37876275358510336      |     0.37876275358510414      
------------------------------------------------------------------------------------------------------------------------
          bathrooms           |      1.211379590539309       |      1.386049497790998       |      1.3860494977910007      
------------------------------------------------------------------------------------------------------------------------
           bedrooms           |     0.41079449220232983      |     0.40682003376918185      |     0.40682003376918124      
